In [ ]:
%cd ..

In [ ]:
import config
import main
from helpers import *
from models import *

In [ ]:
from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [ ]:
actual_prices = Reader.get_actual_prices(config.actual.path, config.num_weeks)
num_assets = actual_prices.data.shape[1]
predicted_returns = [
    Data(Reader.get_predicted_returns(path.path, config.num_weeks, num_assets), path.label)
    for path in config.raw_prediction_paths
]
predicted_prices = ReportTransformer.get_predicted_prices(actual_prices, predicted_returns)

long_short_combinations = [(long, short) for long in range(num_assets) for short in range(num_assets)
                           if not (long == 0 and short == 0)]

In [ ]:
Reporter.plot_correlation_matrix_of_actual_prices(actual_prices)

In [ ]:
Reporter.plot_standardized_prices(actual_prices, predicted_prices)

In [ ]:
random_results = PortfolioGroup.calculate_random_portfolio_for_predictions(config.random_trials, actual_prices, (5, 0))
all_results = PortfolioGroup.calculate_all_portfolio_for_predictions(actual_prices)
all_model_results = PortfolioGroup.calculate_best_portfolio_for_all_predictions(actual_prices, predicted_returns,
                                                                                long_short_combinations)

best_model_results = ReportTransformer.get_best_models(all_model_results)
best_model_names = [m.label for m in best_model_results]

In [ ]:
best_model_results = ReportTransformer.remove_model_prefixes(best_model_results)
portfolio_values_df = ReportTransformer.create_portfolio_values_df(random_results, all_results, best_model_results)
returns_df = ReportTransformer.get_returns(portfolio_values_df)

In [ ]:
Reporter.plot_portfolio_values_of_random_all_and_deep_learning_models(portfolio_values_df)

In [ ]:
Reporter.print_resulting_portfolio_values_for_each_model(portfolio_values_df)

In [ ]:
Reporter.print_portfolio_metrics(portfolio_values_df)

In [ ]:
Reporter.print_portfolio_asset_selection_accuracies(best_model_results + [random_results])

In [ ]:
Reporter.print_statistical_significance_metrics(returns_df)

In [ ]:
Reporter.plot_returns(returns_df)

In [ ]:
Reporter.plot_correlation_matrix_of_returns(returns_df)

In [ ]:
Reporter.print_test_mapes()

In [ ]:
Reporter.print_stats_between_all_predicted_and_actual_prices(actual_prices, predicted_prices)
Reporter.print_stats_between_all_predicted_and_actual_returns(actual_prices, predicted_returns)

In [ ]:
Reporter.print_mapes_between_actual_and_predicted_model_portfolios(best_model_results)

In [ ]:
Reporter.print_direction_based_metrics(actual_prices, predicted_returns, best_model_names)